In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark as spark
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
import os

conf = SparkConf().setAppName('app').setMaster('local')
sc = SparkContext(conf=conf)
hc = SparkSession(sc)
sqlContext = SQLContext(sc)

In [16]:
data = sqlContext.read.csv("wine.csv", header=True)
data.count()

10000

In [17]:
data = data.dropna(subset=['price'])

In [18]:
data = data.drop_duplicates(["description"])

In [19]:
data.count()

9266

In [20]:
def points2group(score):
    splits = [84, 89, 95]
    for (index, split) in enumerate(splits):
        if score is None:
            return 0
        if score < split:
            return index
    return len(splits)


udfPoints2Group = udf(points2group, IntegerType())
data = data.withColumn("points", data['points'].cast('float'))
data = data.withColumn("group", udfPoints2Group("points"))

In [21]:
train,test = data.randomSplit([0.8, 0.2])

In [22]:
(train.count(),test.count())

(7359, 1907)

In [23]:
train.write.csv('train.csv', header=True)
test.write.csv('test.csv', header=True)